In [69]:
import pandas as pd
import math
import seaborn as sns
import matplotlib.pyplot as plt

In [70]:
# def get_subplot_dimensions(N, max_columns=None):
#     if max_columns is None or N <= max_columns:
#         # If max_columns is not specified or N is less than or equal to max_columns,
#         # then the number of columns required is equal to N.
#         # In this case, the number of rows required is 1.
#         return 1, N

#     # If N is greater than max_columns, we need to calculate the number of rows required.
#     rows = math.ceil(N / max_columns)
#     cols = max_columns

#     return rows, cols

In [71]:
RESULTS_DIR = "/home/kieran/Level5ProjectAudioVAE/src/evaluation/data/metrics"

In [73]:
# effects = ['overdrive', 'multiband', 'ambience', 'delay']

In [86]:
cols = ['dafx', 'pesq', 'mrstft', 'msd', 'sce', 'cfe']

In [92]:
vtck = pd.read_csv(f"{RESULTS_DIR}/vctk.csv", index_col=0)[cols]
df = pd.read_csv(f"{RESULTS_DIR}/daps.csv", index_col=0)[cols]
musdb = pd.read_csv(f"{RESULTS_DIR}/musdb18.csv", index_col=0)[cols]

In [93]:
# max_cols = 3
# rows, cols = get_subplot_dimensions(len(metrics), max_columns=max_cols)
# fig, axs = plt.subplots(rows, cols, figsize=(5*cols,4*rows))

# for i, metric in enumerate(metrics):
#     row_idx, col_idx = divmod(i, cols)

#     if rows == 1:
#             current_ax = axs[col_idx]
#     elif cols == 1:
#         current_ax = axs[row_idx]
#     else:
#         current_ax = axs[row_idx, col_idx]
    
#     sns.boxplot(df, x='dafx', y=metric, ax=current_ax)
#     current_ax.set_xticklabels(current_ax.get_xticklabels(), rotation=45)
#     current_ax.set_title(metric.upper())
    
# plt.tight_layout()
# plt.show()

In [96]:
df = vtck.groupby('dafx').mean()

(df.style.highlight_min(axis=0, props='background-color:yellow;', subset=['mrstft', 'msd', 'sce', 'cfe'])
         .highlight_max(axis=0, props='background-color:red;', subset=['mrstft', 'msd', 'sce', 'cfe'])
         .highlight_max(axis=0, props='background-color:yellow;', subset=['pesq'])
         .highlight_min(axis=0, props='background-color:red;', subset=['pesq']))

,pesq,mrstft,msd,sce,cfe
dafx,,,,,
ambience,2.625376,0.524781,1.629467,155.987263,1.663166
combo,3.403186,1.054947,4.278518,312.940663,5.996565
delay,1.611228,1.077613,2.733780,245.982108,2.475333
dynamics,4.206596,0.700966,3.681431,10.342142,4.638893
flanger,2.386474,0.695547,1.913926,140.322264,1.783682
leslie,3.315956,0.621029,2.320705,111.976451,2.899035
multiband,4.450827,0.491711,2.295264,343.810492,2.447423
overdrive,4.223743,0.639782,3.545668,185.566684,3.445503
ringmod,1.148126,1.620978,3.156850,1786.935301,3.104844


In [96]:
print(vtck.head())

      dafx      pesq    mrstft       msd          sce       cfe      lufs  \
0  ringmod  1.264534  1.240924  1.688044   581.219238  0.378036  1.175730   
1  ringmod  1.097677  1.509358  1.852710  2701.941895  1.242100  1.431181   
2  ringmod  1.106961  1.885739  3.897070  3101.211914  8.329063  0.995755   
3  ringmod  1.335033  0.977367  1.059218   148.408691  2.098587  2.480933   
4  ringmod  1.043208  2.118981  7.017373  3594.831543  9.860069  1.100903   

        rms  
0  2.691025  
1  2.523155  
2  2.120857  
3  5.717728  
4  5.352554  


In [126]:
# group the data by 'dafx' category
grouped_df = vtck.groupby('dafx')

# calculate mean and standard deviation for each group
result = grouped_df.agg(['mean', 'std'])

# flatten the column index
result.columns = result.columns.map('_'.join)

# extract the columns you want to format
result = result[['pesq_mean', 'mrstft_mean', 'msd_mean', 'sce_mean', 'cfe_mean', 'lufs_mean', 'rms_mean',
                 'pesq_std', 'mrstft_std', 'msd_std', 'sce_std', 'cfe_std', 'lufs_std', 'rms_std']]

# # format the values as strings
# formatted = result.apply(lambda x: '{:.3f} (std: {:.3f})'.format(x[0], x[1]))
metrics = ['pesq', 'mrstft', 'msd', 'sce', 'cfe', 'lufs']
# # display the final dataframe
# print(formatted)

In [129]:
[result.iloc[0][[f"{m}_mean", f"{m}_std"]] for m in metrics]
    

[pesq_mean    2.625376
 pesq_std     0.868496
 Name: ambience, dtype: float64,
 mrstft_mean    0.524781
 mrstft_std     0.326934
 Name: ambience, dtype: float64,
 msd_mean    1.629467
 msd_std     1.380610
 Name: ambience, dtype: float64,
 sce_mean    155.987263
 sce_std     191.895103
 Name: ambience, dtype: float64,
 cfe_mean    1.663166
 cfe_std     1.833620
 Name: ambience, dtype: float64,
 lufs_mean    1.560620
 lufs_std     1.891791
 Name: ambience, dtype: float64]

In [115]:
result.iloc[0]

pesq_mean        2.625376
mrstft_mean      0.524781
msd_mean         1.629467
sce_mean       155.987263
cfe_mean         1.663166
lufs_mean        1.560620
rms_mean         3.615515
pesq_std         0.868496
mrstft_std       0.326934
msd_std          1.380610
sce_std        191.895103
cfe_std          1.833620
lufs_std         1.891791
rms_std          4.352336
Name: ambience, dtype: float64